In [1]:
import cv2
import random
import numpy as np
import pickle
from skimage import feature

In [10]:
def get_fourier(img):
    img = cv2.resize(img, dsize=(40, 40), interpolation=cv2.INTER_CUBIC)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)[:, :, 1]
    
    fourier = cv2.dft(np.float32(gray), flags=cv2.DFT_COMPLEX_OUTPUT)
    
    fourier_shift = np.fft.fftshift(fourier)

    magnitude = 20*np.log(cv2.magnitude(fourier_shift[:,:,0],fourier_shift[:,:,1]))
    magnitude = cv2.normalize(magnitude, None, 0, 1, cv2.NORM_MINMAX, cv2.CV_8UC1)
 
    phase = cv2.phase(fourier_shift[:,:,0],fourier_shift[:,:,1])
    phase = cv2.normalize(phase, None, 0, 1, cv2.NORM_MINMAX, cv2.CV_8UC1)


    
    return magnitude, phase

def calc_lbp(img, p = 24, r = 8):
    if len(img.shape) != 2:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    else:
        gray = img.copy()
    lbp = feature.local_binary_pattern(gray, p, r, method="uniform")
    # plt.imshow(lbp, cmap="gray")
    # plt.figure()
    hist, _ = np.histogram(lbp.ravel(), bins = p)

    hist = hist.astype("float")
    hist /= hist.sum() 
    return hist

def color_hist(img):
    hsv_img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    
    hists = []
    maxes = (180, 256, 256)
    for i in range(3):
        hists.append(cv2.calcHist([hsv_img[:, :, i]],[0],None,[maxes[i]],[0,maxes[i]]))
    return hists

def fe(img):
    C = calc_lbp(img)
    D = color_hist(img)
    return np.concatenate([
        C, 
        D[0][:,0], 
        D[1][:,0], 
        D[2][:,0],
    ])

def ft(img):
    A, B = get_fourier(img)
    return np.concatenate([A[10:20, 0:20].flatten(), A[0:10, 10:20].flatten(), B[10:20, 0:20].flatten(), B[0:10, 10:20].flatten()])

fe_classifier = None
with open('feature_extrct.pkl','rb') as f:
    fe_classifier = pickle.load(f)

ft_classifier = None
with open('fourier_transform.pkl','rb') as f:
    ft_classifier = pickle.load(f)

def process(image):
    fe_features = fe(image)
    ft_features = ft(image)
    return fe_classifier.predict([fe_features])[0], ft_classifier.predict([ft_features])[0]


In [13]:
paths = []
with open('dataset.txt') as ds:
  for f in ds:
    paths.append(f[:-1])

for file_path in paths:
  cap = cv2.VideoCapture(file_path)
  totalFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
  randomFrameNumber=random.randint(0, totalFrames)
  cap.set(cv2.CAP_PROP_POS_FRAMES,randomFrameNumber)
  success, image = cap.read()
  if not success:
    print ("reading random frame failed")
    continue
  results = process(image)
  print(file_path, results)

../data/VID_20240706_120044-m.mp4 (array([0]), array([1]))
../data/VID_20240706_120108-m.mp4 (array([0]), array([1]))
../data/VID_20240706_120134-m.mp4 (array([0]), array([1]))
../data/VID_20240707_000026-mv.mp4 (array([0]), array([1]))
../data/VID_20240707_000047-mv.mp4 (array([0]), array([1]))
../data/VID_20240707_000109-mv.mp4 (array([0]), array([1]))
